In [3]:
import requests
import getpass

# AUTHORIZATION HEADERS - token oculto
GITHUB_TOKEN = getpass.getpass("Enter your GitHub token: ")

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}

# Endpoint: Buscar repos públicos con la palabra "data"
url = "https://api.github.com/search/repositories?q=data&per_page=5"

# Hacer la solicitud
response = requests.get(url, headers=headers)

# Mostrar resultado
print("Status code:", response.status_code)
print("JSON Response:")
print(response.json())



Enter your GitHub token: ··········
Status code: 200
JSON Response:
{'total_count': 5692589, 'incomplete_results': False, 'items': [{'id': 17829481, 'node_id': 'MDEwOlJlcG9zaXRvcnkxNzgyOTQ4MQ==', 'name': 'data', 'full_name': 'fivethirtyeight/data', 'private': False, 'owner': {'login': 'fivethirtyeight', 'id': 6267336, 'node_id': 'MDEyOk9yZ2FuaXphdGlvbjYyNjczMzY=', 'avatar_url': 'https://avatars.githubusercontent.com/u/6267336?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/fivethirtyeight', 'html_url': 'https://github.com/fivethirtyeight', 'followers_url': 'https://api.github.com/users/fivethirtyeight/followers', 'following_url': 'https://api.github.com/users/fivethirtyeight/following{/other_user}', 'gists_url': 'https://api.github.com/users/fivethirtyeight/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/fivethirtyeight/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/fivethirtyeight/subscriptions', 'organizations_url': 'https://a

In [ ]:
# Obtener el contenido del directorio raíz del repositorio
repo_owner = "octocat"
repo_name = "Hello-World"
path = ""  # carpeta raíz

contents_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{path}"
response = requests.get(contents_url, headers=headers)

print("Status code:", response.status_code)

# Mostrar archivos y carpetas solo si el request fue exitoso
if response.status_code == 200:
    contents = response.json()
    for item in contents:
        print(f"📄 Name: {item['name']}")
        print(f"📂 Type: {item['type']}")
        print(f"🔗 URL: {item['html_url']}")
        print("-" * 40)
else:
    print(f"Error {response.status_code}: {response.json()}")



Status code: 200
📄 Name: README
📂 Type: file
🔗 URL: https://github.com/octocat/Hello-World/blob/master/README
----------------------------------------


In [ ]:
# Buscar repositorios públicos con la palabra clave "data"
search_url = "https://api.github.com/search/repositories"
params = {
    "q": "data",
    "per_page": 5  # para no hacer muchas requests
}

response = requests.get(search_url, headers=headers, params=params)
if response.status_code == 200:
    repos = response.json()["items"]
    for repo in repos:
        print(f"{repo['full_name']} (⭐ {repo['stargazers_count']})")
else:
    print(f"Error: {response.status_code}")


fivethirtyeight/data (⭐ 17120)
emberjs/data (⭐ 3078)
GoogleTrends/data (⭐ 4702)
GSA/data (⭐ 2178)
aptnotes/data (⭐ 1730)


In [ ]:
# Ver commits de un repositorio de ejemplo
owner = "octocat"
repo = "Hello-World"
commits_url = f"https://api.github.com/repos/{owner}/{repo}/commits"

response = requests.get(commits_url, headers=headers)
if response.status_code == 200:
    commits = response.json()
    for commit in commits[:5]:
        print(f"{commit['sha'][:7]}: {commit['commit']['message']}")
else:
    print(f"Error: {response.status_code}")


7fd1a60: Merge pull request #6 from Spaceghost/patch-1

New line at end of file.
7629413: New line at end of file. --Signed off by Spaceghost
553c207: first commit


In [1]:
def get_paginated_results(url, params=None):
    results = []

    while url:
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()  # Lanza excepción si status_code >= 400

            data = response.json()
            # Si es lista, la agregamos directamente. Si es objeto, lo envolvemos en lista
            results.extend(data if isinstance(data, list) else [data])

            # Buscar si hay otra página
            link = response.headers.get("Link", "")
            next_url = None
            if 'rel="next"' in link:
                for part in link.split(','):
                    if 'rel="next"' in part:
                        next_url = part[part.find("<")+1:part.find(">")]
                        break
            url = next_url
            params = None

        except requests.exceptions.HTTPError as http_err:
            print(f"❌ HTTP error occurred: {http_err} - Status code: {response.status_code}")
            break
        except requests.exceptions.RequestException as req_err:
            print(f"⚠️ Request error: {req_err}")
            break
        except Exception as e:
            print(f"🔴 Unexpected error: {e}")
            break

    return results



In [4]:
# Usamos la función para obtener commits de un repositorio
repo_owner = "octocat"
repo_name = "Hello-World"
commits_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/commits"

commits = get_paginated_results(commits_url)
print("Total commits obtenidos:", len(commits))

# Mostrar los primeros 3 commits
for c in commits[:3]:
    print(f"{c['sha'][:7]}: {c['commit']['message']}")


Total commits obtenidos: 3
7fd1a60: Merge pull request #6 from Spaceghost/patch-1

New line at end of file.
7629413: New line at end of file. --Signed off by Spaceghost
553c207: first commit
